# Models

In [2]:
from qilisdk.common.variables import BinaryVariable, Bitwise, Domain, SpinVariable, Variable

x = Variable("x", domain=Domain.REAL, bounds=(1, 2), encoding=Bitwise, precision=1e-1)
s = SpinVariable("s")
b = BinaryVariable("b")

In [3]:
x.to_binary()

(0.1) * x(0) + (0.2) * x(1) + (0.4) * x(2) + (0.30000000000000004) * x(3) + (1.0)

In [4]:
x.evaluate([0, 1, 0, 0])

1.2

## Building Terms from Variables

In [5]:
t1 = 2 * x + 3
print("t1: ", t1)
t2 = 3 * x**2 + 2 * x + 4
print("t2: ", t2)
t3 = 2 * x + b - 1
print("t3: ", t3)
t4 = t1 - t2
print("t4: ", t4)

t1:  (2) * x + (3)
t2:  (3) * (x^2) + (2) * x + (4)
t3:  (2) * x + b + (-1)
t4:  (-1.0) + (-3.0) * (x^2)


In [6]:
t3.evaluate({x: 1.5, b: 0})

2.0

In [7]:
from qilisdk.common.variables import LT

LT(2 * x - 1, 1)

(2) * x < (2.0)

In [8]:
from qilisdk.common.model import Model, ObjectiveSense

model = Model("example_model")

model.set_objective(2 * x + 3, label="obj", sense=ObjectiveSense.MINIMIZE)

print(model)

Model name: example_model 
objective (obj): 
	 minimize : 
	 (2) * x + (3) 

subject to the encoding constraint/s: 
	 x_upper_bound_constraint: x <= (2) 
	 x_lower_bound_constraint: x >= (1) 

With Lagrange Multiplier/s: 
	 x_upper_bound_constraint : 100 
	 x_lower_bound_constraint : 100 



In [9]:
model.set_lagrange_multiplier("x_upper_bound_constraint", 1)

print(model)

Model name: example_model 
objective (obj): 
	 minimize : 
	 (2) * x + (3) 

subject to the encoding constraint/s: 
	 x_upper_bound_constraint: x <= (2) 
	 x_lower_bound_constraint: x >= (1) 

With Lagrange Multiplier/s: 
	 x_upper_bound_constraint : 1 
	 x_lower_bound_constraint : 100 



In [10]:
model.add_constraint("test_constraint", LT(x, 1.5), lagrange_multiplier=10)
print(model)

Model name: example_model 
objective (obj): 
	 minimize : 
	 (2) * x + (3) 

subject to the constraint/s: 
	 test_constraint: x < (1.5) 

subject to the encoding constraint/s: 
	 x_upper_bound_constraint: x <= (2) 
	 x_lower_bound_constraint: x >= (1) 

With Lagrange Multiplier/s: 
	 x_upper_bound_constraint : 1 
	 x_lower_bound_constraint : 100 
	 test_constraint : 10 



In [11]:
model.evaluate({x: 1.4})

{'obj': 5.8, 'test_constraint': 0.0}

In [12]:
model.evaluate({x: 2})

{'obj': 7.0, 'test_constraint': 10.0}

## QUBO

In [13]:
from qilisdk.common.model import QUBO, ObjectiveSense

model = QUBO("example_qubo_model")
model.set_objective(2 * b + 3, label="obj", sense=ObjectiveSense.MINIMIZE)

model.add_constraint(
    "qubo_constraint", LT(2 * b, 1), lagrange_multiplier=10, penalization="slack", transform_to_qubo=True
)

print(model)

Model name: example_qubo_model 
objective (obj): 
	 minimize : 
	 (2) * b + (3) 

subject to the constraint/s: 
	 qubo_constraint: (4.0) * (b * qubo_constraint_slack(0)) + (-1.0) * qubo_constraint_slack(0) == (-1.0) 

With Lagrange Multiplier/s: 
	 qubo_constraint : 10 



In [14]:
print(model.qubo_objective)

obj: (2) * b + (13.0) + (40.0) * (b * qubo_constraint_slack(0)) + (-10.0) * qubo_constraint_slack(0)


In [15]:
from qilisdk.common.model import QUBO, ObjectiveSense

model = QUBO("example_qubo_model")
model.set_objective(2 * b + 3, label="obj", sense=ObjectiveSense.MINIMIZE)

model.add_constraint(
    "qubo_constraint",
    LT(2 * b, 1),
    lagrange_multiplier=1,
    penalization="unbalanced",
    parameters=[1, 2],
    transform_to_qubo=True,
)

print(model)

Model name: example_qubo_model 
objective (obj): 
	 minimize : 
	 (2) * b + (3) 

subject to the constraint/s: 
	 qubo_constraint: (2.0) * b == (-1.0) 

With Lagrange Multiplier/s: 
	 qubo_constraint : 1 



In [16]:
model.qubo_objective

obj: (4.0) * b + (4.0)